In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.output_parsers import StructuredOutputParser, PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tempfile

from typing_extensions import TypedDict, Literal
from pydantic import BaseModel, Field
from typing import Annotated, List, Optional, Union, Dict, Any
from langgraph.graph import StateGraph, START, END


from langchain_community.document_loaders import PyPDFLoader
import camelot

import re, json



### 웹에서 처리

In [37]:
from langchain_community.document_loaders import PyPDFLoader
import camelot

pdf_path = "./bin_pdf_data/BIN_업무협조_요청_VAN_20250801.pdf"

loader = PyPDFLoader(pdf_path)
pages = loader.load() 

# 1) 텍스트 페이지 합치기 (너무 길면 샘플링/요약 후 단계적 호출)
pdf_text = "\n\n".join([f"[p{idx+1}] {d.page_content}" for idx, d in enumerate(pages)])
print('pdf_text: ', pdf_text)

# 2)표 
def df_to_csv(df):
    return df.to_csv(index=False)

# 표 로드 (텍스트 기반 PDF에서만)
tables = camelot.read_pdf(pdf_path, pages="all", flavor="lattice")  # or 'stream'
print(len(tables))
dfs = [t.df for t in tables]  # 각 표를 pandas DataFrame으로
pdf_bin = "\n\n".join([df_to_csv(df) for df in dfs]) if len(dfs) else "N/A"
print(pdf_bin)


pdf_text:  [p1] 정산팀-40-00083
순번 BIN 자체카드
구  분 발급사 기관코드 브랜드 회원구분 카드구분 카드등급 전건대행 신용대행 건별대행 할부가능
여부
2069 9407-57 자체 헥토파이낸셜 57 LOCAL 개인 체크 플러스 대행불가 거래중계 대행불가 할부불가
2070 9200-57 자체 헥토파이낸셜 57 LOCAL 개인+기업 체크 플러스 대행불가 거래중계 대행불가 할부불가
2071 5302-0035 자체 헥토파이낸셜 57 MC 개인 체크 플러스 대행불가 거래중계 대행불가 할부불가
2072 9483-51 자체 비씨카드 50 LOCAL 개인 신용 PT 대행가능 대행가능 대행불가 할부가능
2073 9483-52 자체 비씨카드 50 LOCAL 기업 신용 PT 대행가능 대행가능 대행불가 할부불가
2074 5465-17 자체 비씨카드 50 MC 개인 신용 PT 대행가능 대행가능 대행불가 할부가능
2075 5483-64 자체 비씨카드 50 MC 기업 신용 PT 대행가능 대행가능 대행불가 할부불가
2076 5485-27 자체 수협은행 07 MC 개인 신용 PT 대행가능 대행가능 대행불가 할부가능
 * 신규고객사 "헥토파이낸셜" 매출표 표기 : 헥토카드
ㅇ 일정에 따른 BIN번호 전산반영 및 회신
ㅇ BIN자체관리 가맹점으로 BIN번호 재통지 및 반영여부 확인 요망
  참      조 : 신용카드 담당 부서장
비씨카드주식회사
서울시 중구 을지로 170 을지트윈타워
담당자 : 정산팀 이성희 대리, T: 02)520-8373, E: ssung@bccard.com
  문서번호 : 2025-08-01
  수      신 : 수신처 참조
  제      목 : [비씨카드] BIN 관련 업무협조 요청
1. 귀 사(원)의 무궁한 발전을 기원합니다.
2. 당사 신규 BIN(Bank Identification Number) 생성에 따라 다음과 같이 요청하오니 협조하여 주시기 바랍니다.
ㅇ 할부대행은 6개월 이내에서 가능합니다.
3. 내  용
      가. 신규 BI

### PDF에서 카드 BIN 데이터 추출

In [38]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal
from typing import List, Optional

# PDF 표 구조 
class BinRow(BaseModel):
    bin: str  = Field(description="BIN번호(8자리 또는 6자리)")                                                      # BIN번호 "8자리(또는 6자리) BIN 문자열"
    issuer: str = Field(description="발급사")                                                                     # 발급사
    instCd: str = Field(description="기관코드")                                                                   # 기관코드
    brand: str  = Field(description="브랜드사")                                                                   # 브랜드사
    memberType: Literal["개인","법인","개인+기업"] = Field(description="회원구분")                                   # 회원구분
    cardType:   Literal["체크", "신용", "선불"] = Field(description="카드구분")                                     # 카드구분
    cardRank:   Literal["플러스", "PT", "골드", "체크", "일반", "우량"] = Field(description="대행한도등급")           # 대행한도등급
    agentFull:  Literal["Y","N"]  = Field(description="전건대행")                                                 # 전건대행
    agentCredit: Literal["Y","N"] = Field(description="신용대행")                                                 # 신용대행
    agentPerTxn: Literal["Y","N"] = Field(description="건별대행")                                                 # 건별대행    
    installmentYN: Literal["Y","N"] = Field(description="할부 가능여부")                                           # 할부 가능여부

class ExtractionResult(BaseModel):
    rows: List[BinRow]  = []
    notes: Optional[str]


In [39]:
#parser = PydanticOutputParser(pydantic_object=ExtractionResult)

#print(parser.get_format_instructions())

from langchain_core.language_models import LLM


llm = ChatOpenAI(model="gpt-4.1-mini")
structured_llm = llm.with_structured_output(ExtractionResult)


system = """
당신은 카드 BIN 데이터 추출 보조자입니다.
다음 규칙으로만 JSON을 생성하세요:
- JSON 스키마를 엄격히 준수
- 불확실 시 '?' 사용
- 근거 문맥을 source_text에 200자 이내로 첨부
- 페이지 번호를 page_ref에 기록
"""

human = """PDF에서 추출한 BIN번호 관련 dataframe이 아래에 제공됩니다.
BIN번호, 발급사, 기관코드, 브랜드, 회원구분, 카드구분, 카드등급, 완전대행, 신용대행, 건별대행, 할부가능여부 모두 표준화해 rows 배열로 반환하세요.
텍스트:
{pdf_text}

표(있다면 CSV로 직렬화됨):
{pdf_bin}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", human)
])

pdf_ingestion_chain = prompt | structured_llm
result = pdf_ingestion_chain.invoke({
    "pdf_text": pdf_text[:100000],  # 토큰 초과 방지
    "pdf_bin": pdf_bin[:50000]
    #"format_instructions": parser.get_format_instructions()
})

In [40]:
result.rows

[BinRow(bin='940757', issuer='헥토파이낸셜', instCd='57', brand='LOCAL', memberType='개인', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='Y', agentPerTxn='N', installmentYN='N'),
 BinRow(bin='920057', issuer='헥토파이낸셜', instCd='57', brand='LOCAL', memberType='개인+기업', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='Y', agentPerTxn='N', installmentYN='N'),
 BinRow(bin='53020035', issuer='헥토파이낸셜', instCd='57', brand='MC', memberType='개인', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='Y', agentPerTxn='N', installmentYN='N'),
 BinRow(bin='948351', issuer='비씨카드', instCd='50', brand='LOCAL', memberType='개인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='Y'),
 BinRow(bin='948352', issuer='비씨카드', instCd='50', brand='LOCAL', memberType='법인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='N'),
 BinRow(bin='546517', issuer='비씨카드', instCd='50', brand='MC', memberType='개인', cardT

### BIN 관련 요청 및 DB 조회 필요 여부 검사

In [41]:
# BIN관련 요청 && DB 조회 필요 여부 검사
class NeedDBLookup(BaseModel):
    about_bin: Literal["Y", "N"] = Field(description="BIN 관련 요청인지 여부(Y or N)")
    need_db_lookup: Literal["Y", "N"] = Field(description="DB 조회가 필요한지 여부(Y or N)")
    reason: str = Field(description="DB 조회가 필요하거나 필요하지 않은 이유")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 
structured_llm = llm.with_structured_output(NeedDBLookup)

system = """
당신은 사용자의 질문을 분석하여 BIN 관련 요청인지 여부와 DB 조회가 필요한지 여부를 판단하는 역할을 합니다.

다음과 같은 경우에는 BIN 관련 요청이 아닙니다:
- PDF 텍스트에 BIN 관련 내용이 없는 경우
- 일반적인 대화나 인사
- 개인적인 의견을 묻는 질문
- 창의적인 내용 생성 요청
- 간단한 계산이나 논리적 추론만 필요한 질문

다음과 같은 경우에는 DB 조회가 필요하지 않습니다:
- PDF에 존재하는 BIN의 정보만을 물어보는 경우
- 기존에 존재하는 BIN의 정보를 물어보지 않는 경우
- 일반적인 대화나 인사
- 개인적인 의견을 묻는 질문
- 창의적인 내용 생성 요청
- 간단한 계산이나 논리적 추론만 필요한 질문
- PDF에 있는 내용만 물어보는 경우

다음과 같은 경우에는 DB 조회가 필요합니다:
- PDF에 있는 BIN 정보를 검증해달라고 요청한 경우
- PDF가 아닌, 기존에 존재하는 BIN 정보를 물어보는 경우

또한, 질문은 입력되지 않지만 PDF 텍스트가 존재하며, PDF에 BIN정보가 존재한다면 반드시 DB 조회가 필요합니다.

질문을 분석하고 문서 검색이 필요한지 여부와 그 이유를 제공해주세요.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("user", "질문:{question}"),
    ("user", "PDF 텍스트:{pdf_text}"),
    ("user", "PDF에 있는 BIN 정보:{pdf_ingestion}")
])

need_db_loopup_chain = prompt | structured_llm


In [42]:
pdf_text = """
정산팀-40-00083\n순번 BIN 자체카드\n구  분 발급사 기관코드 브랜드 회원구분 카드구분 카드등급 전건대행 신용대행 건별대행 할부가능\n여부\n2069 9407-57 자체 헥토파이낸셜 57 LOCAL 개인 체크 플러스 대행불가 거래중계 대행불가 할부불가\n2070 9200-57 자체 헥토파이낸셜 57 LOCAL 개인+기업 체크 플러스 대행불가 거래중계 대행불가 할부불가\n2071 5302-0035 자체 헥토파이낸셜 57 MC 개인 체크 플러스 대행불가 거래중계 대행불가 할부불가\n2072 9483-51 자체 비씨카드 50 LOCAL 개인 신용 PT 대행가능 대행가능 대행불가 할부가능\n2073 9483-52 자체 비씨카드 50 LOCAL 기업 신용 PT 대행가능 대행가능 대행불가 할부불가\n2074 5465-17 자체 비씨카드 50 MC 개인 신용 PT 대행가능 대행가능 대행불가 할부가능\n2075 5483-64 자체 비씨카드 50 MC 기업 신용 PT 대행가능 대행가능 대행불가 할부불가\n2076 5485-27 자체 수협은행 07 MC 개인 신용 PT 대행가능 대행가능 대행불가 할부가능\n * 신규고객사 "헥토파이낸셜" 매출표 표기 : 헥토카드\nㅇ 일정에 따른 BIN번호 전산반영 및 회신\nㅇ BIN자체관리 가맹점으로 BIN번호 재통지 및 반영여부 확인 요망\n  참      조 : 신용카드 담당 부서장\n비씨카드주식회사\n서울시 중구 을지로 170 을지트윈타워\n담당자 : 정산팀 이성희 대리, T: 02)520-8373, E: ssung@bccard.com\n  문서번호 : 2025-08-01\n  수      신 : 수신처 참조\n  제      목 : [비씨카드] BIN 관련 업무협조 요청\n1.\xa0귀 사(원)의 무궁한 발전을 기원합니다.\n2.\xa0당사 신규 BIN(Bank Identification Number) 생성에 따라 다음과 같이 요청하오니 협조하여 주시기 바랍니다.\nㅇ 할부대행은 6개월 이내에서 가능합니다.\n3. 내  용\n      가. 신규 BIN 등록\n     ㅇ 다음의 신규 BIN은 2025.08.11(월)까지 전산반영 요청 (개발기 등록 포함)\nㅇ 자체카드구분이 “자체브랜드”인 경우에는 ‘비씨카드’가 아니라 해당 \'발급사의 자체카드\'로 등록하여야 합니다.\nㅇ 매출표 표기는 [발급사명 + 거래형태]로 표기하여 주시기 바랍니다. (Hybrid카드 반영).\n   - 체크카드는 거래형태 \'체크\', 신용카드는 거래형태 \'신용\'으로 표기\nㅇ 기관코드는 비씨카드에서 관리하는 은행 및 카드사 코드입니다.\n 4. 요청사항\n한국정보통신㈜, ㈜케이에스넷, 나이스정보통신㈜, (사)금융결제원, 한국신용카드결제㈜, KIS정보통신㈜, 한국결제네트워크(유), ㈜코밴, ㈜나이스페이먼츠,\n㈜스마트로, ㈜다우데이타, NHNKCP(주), 코레일네트웍스㈜, ㈜섹타나인, 브이피㈜, 토스페이먼트(주) , ㈜하렉스인포텍, ㈜티머니, 농협경제지주,\n㈜케이지이니시스, ㈜이마트, ㈜이랜드, ㈜현대홈쇼핑, ㈜신세계아이앤씨 , ㈜현대백화점, ㈜티비허브  (26개)\n비 씨 카 드 주 식 회 사\n정 산 팀 장\n
"""

question = "PDF에 있는 BIN 정보를 검증해주세요"

pdf_bin_info = """
[{'card_bin': '920057', 'iss_inst_cd': '0100', 'stip_lmt_lvl_cd': '00', 'psn_corp_card_cl': '1', 'card_brand_cd': 'L', 'card_typ_cd': '1', 'mbr_cmpy_cd': '057', 'ist_psbl_yn': 'N', 'stip_psbl_yn': 'N', 'stip_ist_psbl_yn': 'N', 'wgt_cl': 'N'}]\nVIP 콜센터 전화번호는 1566-7890입니다.\n[BinRow(bin='940757', issuer='헥토파이낸셜', instCd='57', brand='LOCAL', memberType='개인', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='N', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='920057', issuer='헥토파이낸셜', instCd='57', brand='LOCAL', memberType='개인+기업', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='N', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='53020035', issuer='헥토파이낸셜', instCd='57', brand='MC', memberType='개인', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='N', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='948351', issuer='비씨카드', instCd='50', brand='LOCAL', memberType='개인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='Y'),\n BinRow(bin='948352', issuer='비씨카드', instCd='50', brand='LOCAL', memberType='법인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='546517', issuer='비씨카드', instCd='50', brand='MC', memberType='개인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='Y'),\n BinRow(bin='548364', issuer='비씨카드', instCd='50', brand='MC', memberType='법인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='548527', issuer='수협은행', instCd='07', brand='MC', memberType='개인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='Y')]
"""

need_db_loopup_chain.invoke(
    {"question": question,
     "pdf_text":  pdf_text, 
     "pdf_ingestion": result.rows
    }
)

NeedDBLookup(about_bin='Y', need_db_lookup='Y', reason='PDF에 있는 BIN 정보를 검증해달라고 요청하였기 때문에 DB 조회가 필요합니다.')

### DB 조회

In [43]:
class DBRow(BaseModel):
    card_bin:str = Field(description="BIN번호(8자리 또는 6자리)")   
    iss_inst_cd: str = Field(description="발급사")
    stip_lmt_lvl_cd: str = Field(description="대행한도등급")
    psn_corp_card_cl: str = Field(description="회원구분")
    card_brand_cd: str = Field(description="브랜드")
    card_typ_cd: str = Field(description="카드구분")
    mbr_cmpy_cd: str = Field(description="기관코드")
    ist_psbl_yn: str = Field(description="할부가능여부")
    stip_psbl_yn: str = Field(description="대행가능여부")
    stip_ist_psbl_yn: str = Field(description="할부대행 가능여부")
    wgt_cl: str = Field(description="건별대행 가능능여부")

In [44]:
from sqlalchemy import create_engine, text, bindparam  
from sqlalchemy.engine import Engine, Result

engine_url = os.getenv("ORACLE_ENGINE_URL")
engine: Engine = create_engine(engine_url, pool_pre_ping=True)  # 필요 시 echo=True


bins = [row.bin for row in result.rows]
print('bins: ', bins)
stmt = text(
    """
    SELECT card_bin, iss_inst_cd, stip_lmt_lvl_cd, psn_corp_card_cl, card_brand_cd, card_typ_cd, mbr_cmpy_cd, ist_psbl_yn, stip_psbl_yn, stip_ist_psbl_yn, wgt_cl
    FROM INF_CARD_BIN
    WHERE 1=1
    AND TRIM(CARD_BIN) IN :bins
    """
).bindparams(bindparam("bins", expanding=True))
params = {
    "bins": bins
}

with engine.connect() as conn:
    select_result: Result = conn.execute(stmt, params)
    dict_rows: List[Dict[str, Any]] = [dict(r._mapping) for r in select_result.fetchall()]
    model_rows: List[DBRow] = [DBRow(**r) for r in dict_rows]

print(len(dict_rows))
print(dict_rows)
print(len(model_rows))
print(model_rows)

bins:  ['940757', '920057', '53020035', '948351', '948352', '546517', '548364', '548527']
7
[{'card_bin': '53020035', 'iss_inst_cd': '0100', 'stip_lmt_lvl_cd': '00', 'psn_corp_card_cl': '1', 'card_brand_cd': 'M', 'card_typ_cd': '1', 'mbr_cmpy_cd': '057', 'ist_psbl_yn': 'N', 'stip_psbl_yn': 'N', 'stip_ist_psbl_yn': 'N', 'wgt_cl': 'N'}, {'card_bin': '548364', 'iss_inst_cd': '0100', 'stip_lmt_lvl_cd': '21', 'psn_corp_card_cl': '2', 'card_brand_cd': 'M', 'card_typ_cd': '0', 'mbr_cmpy_cd': '050', 'ist_psbl_yn': 'N', 'stip_psbl_yn': 'Y', 'stip_ist_psbl_yn': 'N', 'wgt_cl': 'Y'}, {'card_bin': '548527', 'iss_inst_cd': '0202', 'stip_lmt_lvl_cd': '13', 'psn_corp_card_cl': '1', 'card_brand_cd': 'M', 'card_typ_cd': '0', 'mbr_cmpy_cd': '007', 'ist_psbl_yn': 'Y', 'stip_psbl_yn': 'Y', 'stip_ist_psbl_yn': 'Y', 'wgt_cl': 'Y'}, {'card_bin': '920057', 'iss_inst_cd': '0100', 'stip_lmt_lvl_cd': '00', 'psn_corp_card_cl': '1', 'card_brand_cd': 'L', 'card_typ_cd': '1', 'mbr_cmpy_cd': '057', 'ist_psbl_yn': 'N',

### Compare

In [45]:
# 얕은 JSON 타입 (재귀 제거: 안정적인 스키마 생성)
JsonScalar = Union[str, int, float, bool, None]
JsonFlat   = Union[
    JsonScalar,
    Dict[str, JsonScalar],
    List[JsonScalar],
    List[Dict[str, JsonScalar]],
]

class FieldResult(BaseModel):
    ok: bool = Field(..., description="규칙 적합 여부")
    db: JsonFlat = Field(..., description="DB 측 비교값(정규화 후)")
    pdf: JsonFlat = Field(..., description="PDF 측 비교값(정규화 후)")
    note: Optional[str] = Field(None, description="설명/근거")

# ✅ Dict 대신 리스트로(맵 제약 회피)
class ResultEntry(BaseModel):
    key: str = Field(..., description="필드 키 (예: 'card_bin↔bin')")
    result: FieldResult

class ValidationReport(BaseModel):
    # 전 필드 required (기본값/Optional 제거)
    all_passed: bool = Field(..., description="전체 통과 여부")
    results: List[ResultEntry] = Field(..., description="필드별 결과 리스트")
    mismatches: List[str] = Field(..., description="불일치 키 목록")

class PairResult(BaseModel):
    bin: str = Field(..., description="매칭 기준 BIN")
    report: Optional[ValidationReport] = Field(None, description="LLM 비교 리포트 (둘 다 있으면 생성)")
    status: Literal["MATCHED", "PDF_ONLY", "DB_ONLY"] = Field(..., description="매칭 상태")
    pdf_row: Optional[BinRow] = None
    db_row: Optional[DBRow] = None

class BatchValidationReport(BaseModel):
    overall_passed: bool = Field(..., description="전체 행이 모두 통과했는지 여부")
    total_pairs: int = Field(..., description="매칭된 쌍(또는 케이스)의 총 개수")
    matched: int = Field(..., description="PDF & DB 모두 존재하는 케이스 수")
    pdf_only: int = Field(..., description="PDF에만 있고 DB엔 없는 케이스 수")
    db_only: int = Field(..., description="DB에만 있고 PDF엔 없는 케이스 수")
    pairs: List[PairResult] = Field(..., description="BIN별 상세 결과")

class PairReport(BaseModel):
    bin: str = Field(..., description="매칭 기준 BIN")
    report: ValidationReport = Field(..., description="해당 BIN 쌍의 검증 리포트")

class BatchLLMResult(BaseModel):
    pair_reports: List[PairReport] = Field(..., description="각 BIN 쌍의 리포트 목록")



In [46]:
#DBRow의 model_rows와 pdf_ingestion의 result.rows를 비교하여 결과를 출력
SYSTEM_RULES = """
너는 카드 BIN 데이터 검증기다.
아래 "검증 규칙"에 따라 DB 값 ↔ PDF 값을 비교하라.
입력은 여러 쌍의 BIN이며, 각 쌍에 대해 ValidationReport를 생성하여 배열로 반환한다.

[검증 규칙]

1. card_bin ↔ bin
   - 단순 문자열 동일성 비교.

2. iss_inst_cd ↔ issuer
   - 매핑표: {{0201: 농협, 0500: 신한, 0202: 수협, 0203: 씨티,
             0301: 제주, 0302: 광주, 0303: 전북, 0100: 비씨}}
   - PDF issuer가 {{농협, 신한, 수협, 씨티, 제주, 광주, 전북, 비씨, BC}} 외면
     "비씨"로 간주하여 비교.

3. stip_lmt_lvl_cd ↔ cardRank
   - 매핑표: {{21: 법인, 13: PT, 17: 골드, 16: 우량, 15: 일반, 00: 플러스}}
   - 단, PDF memberType이 "법인"이면 DB 값은 반드시 21이어야 함.

4. psn_corp_card_cl ↔ memberType
   - 매핑표: {{1: 개인일반, 2: 법인카드}}
   - 단, PDF 값이 "개인+기업"이면 DB 값은 1이어야 함.

5. card_brand_cd ↔ brand
   - 매핑표: {{A: AMEX, J: JCB, L: LOCAL, V: VISA, M: MASTER, MC: MASTER,
             D: DINERS, C: 은련, G: GLOBAL, K: JUST TOUCH, N: CONA}}
   - PDF brand는 대소문자 무시.

6. card_typ_cd ↔ cardType
   - 매핑표: {{0: 신용, 1: 체크, 2: 선불}}
   - "체크카드"도 "체크"로 처리.

7. mbr_cmpy_cd ↔ instCd
   - 앞자리 0 제거 후 정수 비교.

8. ist_psbl_yn ↔ installmentYN
   - Y/N 동일 비교.

9. stip_psbl_yn ↔ agentFull & agentCredit
   - agentFull, agentCredit 둘 다 'Y' 이면 stip_psbl_yn = 'Y'
      위 경우가 아니면 아니면 stip_psbl_yn은 = 'N'.

10. stip_ist_psbl_yn ↔ agentFull & agentCredit & installmentYN
    - agentFull, agentCredit, installmentYN 모두 'Y'이면, stip_ist_psbl_yn = 'Y'
      위 경우가 아니면 stip_ist_psbl_yn = 'N'.

11. wgt_cl ↔ agentPerTxn
    - Y/N 동일 비교.

"""

HUMAN = """
다음은 여러 BIN 쌍의 입력이다. 각 항목은 동일한 BIN에 대한 PDF 행과 DB 행이다.
각 항목마다 ValidationReport를 만들어 배열로 반환하라.

입력:
{pairs_json}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_RULES.strip()),
    ("human", HUMAN.strip())
])

# ===== 모델 바인딩 =====
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
structured_llm = llm.with_structured_output(BatchLLMResult)
# 그냥 JSON 결과만 받도록
batch_chain = prompt | structured_llm
# BIN 정규화 유틸
def norm_bin(s: str) -> str:
    return re.sub(r"[\s\-]", "", s or "").strip()

# ===== 배치 비교 =====
def compare_many_batched(pdf_rows: List[BinRow], db_rows: List[DBRow]) -> List[Dict]:
    pdf_map: Dict[str, BinRow] = {norm_bin(p.bin): p for p in pdf_rows if norm_bin(p.bin)}
    db_map: Dict[str, DBRow]   = {norm_bin(d.card_bin): d for d in db_rows if norm_bin(d.card_bin)}

    matched_keys = [k for k in pdf_map if k in db_map]

    pairs_payload = [
        {"bin": k, "pdf": pdf_map[k].model_dump(), "db": db_map[k].model_dump()}
        for k in matched_keys
    ]

    if not pairs_payload:
        return []

    pairs_json_str = json.dumps({"pairs": pairs_payload}, ensure_ascii=False)

    # LLM 호출 → JSON 문자열 결과
    llm_out = batch_chain.invoke({"pairs_json": pairs_json_str})

    return llm_out

In [47]:
pdf_rows: List[BinRow]  = result.rows # PDF에서 추출된 행들
db_rows:  List[DBRow]  = model_rows  # DB에서 조회된 행들
batch_report = compare_many_batched(pdf_rows, db_rows)
print(batch_report.model_dump())

{'pair_reports': [{'bin': '940757', 'report': {'all_passed': True, 'results': [{'key': 'card_bin↔bin', 'result': {'ok': True, 'db': '940757', 'pdf': '940757', 'note': None}}, {'key': 'iss_inst_cd↔issuer', 'result': {'ok': True, 'db': '0100', 'pdf': '헥토파이낸셜', 'note': 'PDF issuer가 매핑표 외면 비씨로 간주, DB는 0100(비씨)'}}, {'key': 'stip_lmt_lvl_cd↔cardRank', 'result': {'ok': True, 'db': '00', 'pdf': '플러스', 'note': None}}, {'key': 'psn_corp_card_cl↔memberType', 'result': {'ok': True, 'db': '1', 'pdf': '개인', 'note': None}}, {'key': 'card_brand_cd↔brand', 'result': {'ok': True, 'db': 'L', 'pdf': 'LOCAL', 'note': None}}, {'key': 'card_typ_cd↔cardType', 'result': {'ok': True, 'db': '1', 'pdf': '체크', 'note': '체크카드도 체크로 처리'}}, {'key': 'mbr_cmpy_cd↔instCd', 'result': {'ok': True, 'db': '057', 'pdf': '57', 'note': '앞자리 0 제거 후 정수 비교'}}, {'key': 'ist_psbl_yn↔installmentYN', 'result': {'ok': True, 'db': 'N', 'pdf': 'N', 'note': None}}, {'key': 'stip_psbl_yn↔agentFull & agentCredit', 'result': {'ok': True, 'db'

In [48]:
batch_report

BatchLLMResult(pair_reports=[PairReport(bin='940757', report=ValidationReport(all_passed=True, results=[ResultEntry(key='card_bin↔bin', result=FieldResult(ok=True, db='940757', pdf='940757', note=None)), ResultEntry(key='iss_inst_cd↔issuer', result=FieldResult(ok=True, db='0100', pdf='헥토파이낸셜', note='PDF issuer가 매핑표 외면 비씨로 간주, DB는 0100(비씨)')), ResultEntry(key='stip_lmt_lvl_cd↔cardRank', result=FieldResult(ok=True, db='00', pdf='플러스', note=None)), ResultEntry(key='psn_corp_card_cl↔memberType', result=FieldResult(ok=True, db='1', pdf='개인', note=None)), ResultEntry(key='card_brand_cd↔brand', result=FieldResult(ok=True, db='L', pdf='LOCAL', note=None)), ResultEntry(key='card_typ_cd↔cardType', result=FieldResult(ok=True, db='1', pdf='체크', note='체크카드도 체크로 처리')), ResultEntry(key='mbr_cmpy_cd↔instCd', result=FieldResult(ok=True, db='057', pdf='57', note='앞자리 0 제거 후 정수 비교')), ResultEntry(key='ist_psbl_yn↔installmentYN', result=FieldResult(ok=True, db='N', pdf='N', note=None)), ResultEntry(key='s

### 결과 생성

In [49]:
class ExplainResult(BaseModel):
    pdf_bin: str = Field(..., description="pdf에서 추출한 bin 정보표 (마크다운 테이블)")
    db_bin: str = Field(..., description="db에서 조회한 bin 정보표 (마크다운 테이블)")
    explain: str = Field(..., description="결과 설명(한글 요약)")

In [50]:
def md_table_from_rows(rows: List[Dict[str, Any]], headers: List[str]) -> str:
    """
    headers 순서대로 rows의 값을 뽑아 마크다운 테이블을 만듭니다.
    """
    if not rows:
        return "_데이터 없음_"
    # 헤더
    md = "|" + "|".join(headers) + "|\n"
    md += "|" + "|".join(["---"] * len(headers)) + "|\n"
    # 바디
    for r in rows:
        line = []
        for h in headers:
            v = r.get(h, "")
            line.append(str(v) if v is not None else "")
        md += "|" + "|".join(line) + "|\n"
    return md

def pdf_rows_to_md(pdf_rows: List[BinRow]) -> str:
    headers = ["bin","issuer","instCd","brand","memberType","cardType","cardRank","agentFull","agentCredit","agentPerTxn","installmentYN"]
    dict_rows = [r.model_dump() for r in pdf_rows]
    return md_table_from_rows(dict_rows, headers)

def db_rows_to_md(db_rows: List[DBRow]) -> str:
    headers = ["card_bin","iss_inst_cd","stip_lmt_lvl_cd","psn_corp_card_cl","card_brand_cd","card_typ_cd","mbr_cmpy_cd","ist_psbl_yn","stip_psbl_yn","stip_ist_psbl_yn","wgt_cl"]
    dict_rows = [r.model_dump() for r in db_rows]
    return md_table_from_rows(dict_rows, headers)


In [51]:
SYSTEM_MSG = """
너는 카드 BIN 데이터 검증 보고서 작성 보조자다.
아래 입력을 바탕으로 최종 ExplainResult(JSON)만 생성한다.
규칙:
- pdf_bin, db_bin은 이미 제공된 마크다운 테이블 문자열을 그대로 사용한다(수정/재생성 금지).
- explain에는 핵심 요약을 한국어로 작성한다.
  - 총 건수, 통과 건수, 불일치 건수
  - 주요 불일치 필드와 예시(가능하면 3건 이내로)
  - 다음 액션(예: DB 반영 필요, PDF 원문 확인 필요 등)
- 오직 ExplainResult 스키마(JSON) 하나만 출력한다.
"""

HUMAN_TMPL = """
입력 데이터는 다음과 같다.

[PDF 테이블 (마크다운)]
{pdf_table_md}

[DB 테이블 (마크다운)]
{db_table_md}

[검증 결과 요약 입력(JSON)]
- 통과 건수(all_passed=True): {passed_count} / 매칭 건수: {matched_count}
- 불일치 건수(all_passed=False): {failed_count}
- 대표 불일치(최대 {max_samples}건):
{mismatch_samples_md}

[검증 원문(JSON, 필요 시 참고)]
{validation_json_snippet}

요청:
- 위 정보를 종합하여 ExplainResult(JSON)만 생성하라.
- pdf_bin 필드에는 위의 PDF 테이블 문자열을 그대로 넣고,
  db_bin 필드에는 위의 DB 테이블 문자열을 그대로 넣어라.
- explain에는 위 요약을 자연스러운 한국어로 정리하라.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_MSG.strip()),
    ("user", HUMAN_TMPL.strip())
])

# ----------------------------------------
# LLM: 구조화 결과로 ExplainResult 받기
# ----------------------------------------
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
structured_llm = llm.with_structured_output(ExplainResult)
explain_chain = prompt | structured_llm

In [52]:
# 공통 LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
structured_llm = llm.with_structured_output(ExplainResult)

# (A) lookup_only 프롬프트 (DB 없이 설명만)
LOOKUP_SYSTEM = """
너는 BIN 검증 어시스턴트다.
이번 케이스는 DB 조회 없이, PDF에서 추출된 내용과 사용자 질문을 바탕으로 설명만 생성한다.
출력은 ExplainResult(JSON) 하나로 만들되,
- pdf_bin: 제공된 PDF 표 문자열 그대로
- db_bin: "N/A" 로 표기
- explain: 한국어로 사용자 질문에 대한 맥락 있는 답변 + 다음 액션 제안(예: DB 조회 필요 여부)
"""

LOOKUP_HUMAN = """
[사용자 질문]
{question}

[PDF 표(마크다운)]
{pdf_table_md}

요청:
- 위 정보를 바탕으로 ExplainResult(JSON)만 생성하라.
- db_bin은 "N/A"로 두고, explain에 답변을 작성한다.
"""

lookup_prompt = ChatPromptTemplate.from_messages([
    ("system", LOOKUP_SYSTEM.strip()),
    ("user", LOOKUP_HUMAN.strip())
])

lookup_chain = lookup_prompt | structured_llm

In [22]:
  # 1) 표 마크다운 생성
pdf_table_md = pdf_rows_to_md(pdf_rows)
db_table_md  = db_rows_to_md(db_rows)

In [31]:
print(db_table_md)

|card_bin|iss_inst_cd|stip_lmt_lvl_cd|psn_corp_card_cl|card_brand_cd|card_typ_cd|mbr_cmpy_cd|ist_psbl_yn|stip_psbl_yn|stip_ist_psbl_yn|wgt_cl|
|---|---|---|---|---|---|---|---|---|---|---|
|53020035|0100|00|1|M|1|057|N|N|N|N|
|548364|0100|21|2|M|0|050|N|Y|N|Y|
|548527|0202|13|1|M|0|007|Y|Y|Y|Y|
|920057|0100|00|1|L|1|057|N|N|N|N|
|940757|0100|00|1|L|1|057|N|N|N|N|
|948351|0100|13|1|L|0|050|Y|Y|Y|Y|
|948352|0100|21|2|L|0|050|N|Y|N|Y|



In [25]:
 # 2) validation 집계
validation = batch_report.pair_reports
matched_count = len(validation)
passed_count  = sum(1 for v in validation if v.report and v.report.all_passed)
failed_count  = matched_count - passed_count

In [28]:
# 대표 불일치 샘플(최대 5건)
max_samples = 5
mismatches_samples = []
for v in validation:
    if v.report and not v.report.all_passed:
        # 간단 요약: BIN, mismatches(키들)
        mismatches_samples.append({
            "bin": v.bin,
            "mismatches": v.report.mismatches[:10] if v.report.mismatches else [],
        })
    if len(mismatches_samples) >= max_samples:
        break

# 사람이 읽기 쉬운 마크다운 불일치 샘플
if mismatches_samples:
    header = "| BIN | mismatches |\n|---|---|\n"
    body = "\n".join([f"| {s['bin']} | {', '.join(s['mismatches']) or '-'} |" for s in mismatches_samples])
    mismatch_samples_md = header + body
else:
    mismatch_samples_md = "_불일치 샘플 없음_"

# validation 원문은 너무 길 수 있어 일부만
try:
    validation_json_snippet = json.dumps(
        [ {"bin": v.bin, "all_passed": v.report.all_passed, "mismatches": v.report.mismatches}
            for v in validation
        ],
        ensure_ascii=False
    )
    if len(validation_json_snippet) > 8000:
        validation_json_snippet = validation_json_snippet[:8000] + "...(truncated)"
except Exception:
    validation_json_snippet = "_serialization error_"

In [29]:
ExplainResult = explain_chain.invoke({
        "pdf_table_md": pdf_table_md,
        "db_table_md": db_table_md,
        "passed_count": passed_count,
        "matched_count": matched_count,
        "failed_count": failed_count,
        "max_samples": max_samples,
        "mismatch_samples_md": mismatch_samples_md,
        "validation_json_snippet": validation_json_snippet,
    })

In [30]:
ExplainResult

ExplainResult(pdf_bin='|bin|issuer|instCd|brand|memberType|cardType|cardRank|agentFull|agentCredit|agentPerTxn|installmentYN|\n|---|---|---|---|---|---|---|---|---|---|---|\n|940757|헥토파이낸셜|57|LOCAL|개인|체크|플러스|N|Y|N|N|\n|920057|헥토파이낸셜|57|LOCAL|개인+기업|체크|플러스|N|Y|N|N|\n|53020035|헥토파이낸셜|57|MC|개인|체크|플러스|N|Y|N|N|\n|948351|비씨카드|50|LOCAL|개인|신용|PT|Y|Y|N|Y|\n|948352|비씨카드|50|LOCAL|법인|신용|PT|Y|Y|N|N|\n|546517|비씨카드|50|MC|개인|신용|PT|Y|Y|N|Y|\n|548364|비씨카드|50|MC|법인|신용|PT|Y|Y|N|N|\n|548527|수협은행|07|MC|개인|신용|PT|Y|Y|N|Y|', db_bin='|card_bin|iss_inst_cd|stip_lmt_lvl_cd|psn_corp_card_cl|card_brand_cd|card_typ_cd|mbr_cmpy_cd|ist_psbl_yn|stip_psbl_yn|stip_ist_psbl_yn|wgt_cl|\n|---|---|---|---|---|---|---|---|---|---|---|\n|53020035|0100|00|1|M|1|057|N|N|N|N|\n|548364|0100|21|2|M|0|050|N|Y|N|Y|\n|548527|0202|13|1|M|0|007|Y|Y|Y|Y|\n|920057|0100|00|1|L|1|057|N|N|N|N|\n|940757|0100|00|1|L|1|057|N|N|N|N|\n|948351|0100|13|1|L|0|050|Y|Y|Y|Y|\n|948352|0100|21|2|L|0|050|N|Y|N|Y|', explain='총 7건의 BIN 데이터가 PDF와 DB에서 모두 매칭되어

ExplainResult(pdf_bin='|bin|issuer|instCd|brand|memberType|cardType|cardRank|agentFull|agentCredit|agentPerTxn|installmentYN|\n|---|---|---|---|---|---|---|---|---|---|---|\n|940757|헥토파이낸셜|57|LOCAL|개인|체크|플러스|N|Y|N|N|\n|920057|헥토파이낸셜|57|LOCAL|개인+기업|체크|플러스|N|Y|N|N|\n|53020035|헥토파이낸셜|57|MC|개인|체크|플러스|N|Y|N|N|\n|948351|비씨카드|50|LOCAL|개인|신용|PT|Y|Y|N|Y|\n|948352|비씨카드|50|LOCAL|법인|신용|PT|Y|Y|N|N|\n|546517|비씨카드|50|MC|개인|신용|PT|Y|Y|N|Y|\n|548364|비씨카드|50|MC|법인|신용|PT|Y|Y|N|N|\n|548527|수협은행|07|MC|개인|신용|PT|Y|Y|N|Y|', db_bin='|card_bin|iss_inst_cd|stip_lmt_lvl_cd|psn_corp_card_cl|card_brand_cd|card_typ_cd|mbr_cmpy_cd|ist_psbl_yn|stip_psbl_yn|stip_ist_psbl_yn|wgt_cl|\n|---|---|---|---|---|---|---|---|---|---|---|\n|53020035|0100|00|1|M|1|057|N|N|N|N|\n|548364|0100|21|2|M|0|050|N|Y|N|Y|\n|548527|0202|13|1|M|0|007|Y|Y|Y|Y|\n|920057|0100|00|1|L|1|057|N|N|N|N|\n|940757|0100|00|1|L|1|057|N|N|N|N|\n|948351|0100|13|1|L|0|050|Y|Y|Y|Y|\n|948352|0100|21|2|L|0|050|N|Y|N|Y|', explain='총 7건의 BIN 데이터가 PDF와 DB에서 모두 매칭되어 검증을 통과하였습니다. 불일치 건수는 없으며, 대표적인 불일치 사례도 존재하지 않습니다. 다만 일부 항목(wgt_cl와 agentPerTxn) 간의 차이가 있으나, 이는 검증 결과에 영향을 주지 않아 전체적으로 일치하는 것으로 판단됩니다. 따라서 현재 데이터는 신뢰할 수 있으며, 추가적인 DB 반영이나 PDF 원문 확인은 필요하지 않습니다.')

In [ ]:
# LangGraph에서 쓸 상태(State) 정의입
# 그래프가 흘러가면서 공유/업데이트되는 데이터의 스키마를 TypedDict로 정의의
class GraphState(TypedDict):
    question : Annotated[str, "사용자의 질문"]
    pdf_text : Annotated[str, "PDF 텍스트"]
    pdf_bin : Annotated[str, "PDF BIN 정보"]

    pdf_ingestion : Annotated[List[BinRow], "PDF 텍스트  추출 결과"]

    need_db_lookup : Annotated[str, "DB 조회 필요 여부"]
    db_rows : Annotated[List[DBRow], "DB 조회 결과"]
    
    validation : Annotated[List[PairReport], "ValidationReport 결과"]

    explain: Annotated[ExplainResult, "최종 보고서"]

### 노드 정의

In [ ]:
def pdf_ingestion(state:GraphState) -> GraphState:
    print("PDF INGESTION....")
    pdf_text = state["pdf_text"]
    pdf_bin = state["pdf_bin"]

    pdf_ingestion_result = pdf_ingestion_chain.invoke({
        "pdf_text": pdf_text,
        "pdf_bin": pdf_bin
    })
    return {'pdf_ingestion': pdf_ingestion_result}

# 전처리를 진행하고, need_db_lookup 노드로 이동
def need_db_lookup(state:GraphState) -> GraphState:
    print("NEED DB LOOKUP....")
    question = state["question"]
    pdf_text = state["pdf_text"]
    pdf_ingestion = state["pdf_ingestion"]

    need_db_lookup_result = need_db_loopup_chain.invoke({
        "question": question,
        "pdf_text": pdf_text,
        "pdf_ingestion": pdf_ingestion
    })
    return {'about_bin': need_db_lookup_result.about_bin, 'need_db_lookup':need_db_lookup_result.need_db_lookup}

def fetch_db(state:GraphState) -> GraphState:
    print("FETCH DB....")
    bins = [row.bin for row in pdf_ingestion]
    
    stmt = text(
        """
        SELECT card_bin, iss_inst_cd, stip_lmt_lvl_cd, psn_corp_card_cl, card_brand_cd, card_typ_cd, mbr_cmpy_cd, ist_psbl_yn, stip_psbl_yn, stip_ist_psbl_yn, wgt_cl
        FROM INF_CARD_BIN
        WHERE 1=1
        AND TRIM(CARD_BIN) IN :bins
        """
    ).bindparams(bindparam("bins", expanding=True))
    params = {
        "bins": bins
    }

    with engine.connect() as conn:
        select_result: Result = conn.execute(stmt, params)
        dict_rows: List[Dict[str, Any]] = [dict(r._mapping) for r in select_result.fetchall()]
        model_rows: List[DBRow] = [DBRow(**r) for r in dict_rows]

    print(len(dict_rows))
    print(dict_rows)
    print(len(model_rows))
    print(model_rows)
    
    return {'db_rows': model_rows}

def validation(state:GraphState) -> GraphState:
    print("VALIDATION....")
    pdf_ingestion = state["pdf_ingestion"]
    db_rows = state["db_rows"]
    validation_result = compare_many_batched(pdf_rows, db_rows)

    return {'validation': validation_result}

def explain(state:GraphState) -> GraphState:
    print("EXPLAIN....")
    pdf_ingestion = state["pdf_ingestion"]
    db_rows = state["db_rows"]
    validation = state["validation"].pair_reports
    need_db_lookup = state["need_db_lookup"]


    # 1) 표 마크다운 생성
    pdf_table_md = pdf_rows_to_md(pdf_ingestion)
    db_table_md  = db_rows_to_md(db_rows)


    if need_db_lookup == "N":
        # DB 없이 말로만 응답
        question = state[question]
        result = lookup_chain.invoke({
            "question": question,
            "pdf_table_md": pdf_table_md,
        })
        # db_bin은 모델이 "N/A"로 채우도록 지시했음
        return {"explain": result}

    # 2) validation 집계
    matched_count = len(validation)
    passed_count  = sum(1 for v in validation if v.report and v.report.all_passed)
    failed_count  = matched_count - passed_count

    # 대표 불일치 샘플(최대 5건)
    max_samples = 5
    mismatches_samples = []
    for v in validation:
        if v.report and not v.report.all_passed:
            # 간단 요약: BIN, mismatches(키들)
            mismatches_samples.append({
                "bin": v.bin,
                "mismatches": v.report.mismatches[:10] if v.report.mismatches else [],
            })
        if len(mismatches_samples) >= max_samples:
            break

    # 사람이 읽기 쉬운 마크다운 불일치 샘플
    if mismatches_samples:
        header = "| BIN | mismatches |\n|---|---|\n"
        body = "\n".join([f"| {s['bin']} | {', '.join(s['mismatches']) or '-'} |" for s in mismatches_samples])
        mismatch_samples_md = header + body
    else:
        mismatch_samples_md = "_불일치 샘플 없음_"

    # validation 원문은 너무 길 수 있어 일부만
    try:
        validation_json_snippet = json.dumps(
            [ {"bin": v.bin, "all_passed": v.report.all_passed, "mismatches": v.report.mismatches}
              for v in validation
            ],
            ensure_ascii=False
        )
        if len(validation_json_snippet) > 8000:
            validation_json_snippet = validation_json_snippet[:8000] + "...(truncated)"
    except Exception:
        validation_json_snippet = "_serialization error_"

    # 3) LLM 호출 → ExplainResult
    result: ExplainResult = explain_chain.invoke({
        "pdf_table_md": pdf_table_md,
        "db_table_md": db_table_md,
        "passed_count": passed_count,
        "matched_count": matched_count,
        "failed_count": failed_count,
        "max_samples": max_samples,
        "mismatch_samples_md": mismatch_samples_md,
        "validation_json_snippet": validation_json_snippet,
    })

    # 4) state에 저장 후 반환
    return {"explain": result}
    
        

### 조건부 엣지 정의

In [ ]:
# 문서 검색 필요 여부 검사
def decide_need_db_lookup(state:GraphState) -> GraphState:
    print("DECISION NEED RETRIEVAL....")
    about_bin = state["about_bin"]
    need_db_lookup = state["need_db_lookup"]

    if need_db_lookup == "N":
        return 'explain'
    else:
        return 'fetch_db'

### 그래프 생성

In [26]:

workflow = StateGraph(GraphState)

workflow.add_node("pdf_ingestion", pdf_ingestion)
workflow.add_node("need_db_lookup", need_db_lookup)
workflow.add_node("fetch_db", fetch_db)

workflow.add_edge(START, "pdf_ingestion")
workflow.add_edge("pdf_ingestion", "need_db_lookup")
workflow.add_conditional_edges(
    "need_db_lookup",                   # 분기를 만드는 기준 노드 (need_db_lookup 노드)
    decide_need_db_lookup,              # 조건 함수 (state를 보고 다음 노드 이름 리턴)
    {
        'explain': 'explain',           # 조건 함수가 'explain'를 리턴하면 → explain 노드로
        'fetch_db': 'fetch_db'          # 조건 함수가 'fetch_db'를 리턴하면 → fetch_db 노드로
    }
)
workflow.add_edge("fetch_db", "validation")
workflow.add_edg("validation", "explain")

app = workflow.compile()